# Getting started with GitHub Models - Azure AI Inference SDK

## 1. Personal access token

A personal access token is made available in the Codespaces environment in the `GITHUB_TOKEN` environment variable. 



## 2. Install dependencies

In [1]:
%pip install azure-ai-inference --quiet
%pip install python-dotenv --quiet


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.




## 3. Set environment variables and create the client




In [2]:
import os
import dotenv
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

dotenv.load_dotenv()

if not os.getenv("GITHUB_TOKEN"):
    raise ValueError("GITHUB_TOKEN is not set")

github_token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"


# Create a client
client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(github_token),
)


By using the Azure AI Inference SDK, you can easily experiment with different models by modifying the value of `model_name` in the code below. The following models are available in the GitHub Models service:
- AI21 Labs: `AI21-Jamba-Instruct`
- Cohere: `Cohere-command-r`, `Cohere-command-r-plus`
- Meta: `Meta-Llama-3-70B-Instruct`, `Meta-Llama-3-8B-Instruct`, `Meta-Llama-3.1-405B-Instruct`, `Meta-Llama-3.1-70B-Instruct`, `Meta-Llama-3.1-8B-Instruct`
- Mistral AI: `Mistral-large`, `Mistral-large-2407`, `Mistral-Nemo`, `Mistral-small`
- Azure OpenAI: `gpt-4o-mini`, `gpt-4o`
- Microsoft: `Phi-3-medium-128k-instruct`, `Phi-3-medium-4k-instruct`, `Phi-3-mini-128k-instruct`, `Phi-3-mini-4k-instruct`, `Phi-3-small-128k-instruct`, `Phi-3-small-8k-instruct`

In [3]:
# pick one of them
model_name = "gpt-4o-mini"

## 4. Run a basic code sample

This is just calling the `chat.completions` endpoint with a simple prompt.




In [4]:
response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content="What is the capital of Uruguay?"),
    ],
    model=model_name,
    # Optional parameters
    temperature=1.,
    max_tokens=1000,
    top_p=1.    
)

print(response.choices[0].message.content)

The capital of Uruguay is Montevideo.


## 5. Multi-Turn Conversation

This sample demonstrates a multi-turn conversation with the chat completion API.
When using the model for a chat application, you'll need to manage the history of that
conversation and send the latest messages to the model.



In [5]:
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage

messages = [
    SystemMessage(content="You are a helpful assistant."),
    UserMessage(content="What is the capital of Uruguay?"),
    AssistantMessage(content="The capital of Uruguay is Montevideo."),
    UserMessage(content="What about Argentina?"),
]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)

The capital of Argentina is Buenos Aires.


## 6. Using images as inputs

Some models of the GitHub Models service support image inputs. The following image models are available in the GitHub Models service:
 
-  Azure OpenAI: `gpt-4o-mini`, `gpt-4o`

In [ ]:
# Select a model
model_name = "gpt-4o-mini"

To run a chat completion using a local image file, use the following sample.

![image](./sample.png)

> Note: To send it to the service, you'll need to encode the image as **data URI**, which is a string that starts with `data:image/png;base64,` followed by the base64-encoded image. The Azure AI Inference SDK provides classes to help creating a chat completion for such models.

In [7]:
from azure.ai.inference.models import (
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="que muestra esta imagen?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.png",
                        image_format="png",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

La imagen muestra a un pequeño cachorro de color dorado, sentado en el suelo de un interior acogedor. El cachorro parece estar esperando con atención, y frente a él hay un tazón de comida o agua en el suelo. El entorno tiene un aspecto cálido y hogareño, con un piso de madera y algunas piezas de mobiliario de fondo que sugieren un ambiente cómodo. 


## 7. Streaming the response

For a better user experience, you will want to stream the response of the model
so that the first token shows up early and you avoid waiting for long responses.






In [9]:
response = client.complete(
    stream=True,
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content="Give me 5 good reasons why I should not exercise every day."),
    ],
    model=model_name,
)

for update in response:
    if update.choices:
        print(update.choices[0].delta.content or "", end="")

print()



While exercise is beneficial for health, there

KeyboardInterrupt: 

## 8. Tools and Function Calling

Some models of the GitHub Models service support tools. A language model is given a set of tools it can ask the calling program to invoke,
for running specific actions depending on the context of the conversation. This sample demonstrates how to define a function tool and how to act on a request from the model to invoke it.

The following compatible models are available in the GitHub Models service:
- Cohere: `Cohere-command-r`, `Cohere-command-r-plus`
- Mistral AI: `Mistral-large`, `Mistral-large-2407`, `Mistral-Nemo`, `Mistral-small`
- Azure OpenAI: `gpt-4o-mini`, `gpt-4o`

In [10]:
# pick one of them
model_name = "gpt-4o-mini"

In [12]:
import json

# Define a function that returns flight information between two cities (mock implementation)
def get_flight_info(origin_city: str, destination_city: str):
    if origin_city == "Seattle" and destination_city == "Miami":
        return json.dumps(
            {
                "airline": "Iberia",
                "flight_number": "IL123",
                "flight_date": "May 12th, 2023",
                "flight_time": "10:10AM",
            }
        )
    return json.dump({"error": "No flights found between the cities"})


# Define a function tool that the model can ask to invoke in order to retrieve flight information
tool = {
    "type": "function",
    "function": {
        "name": "get_flight_info",
        "description": """Returns information about the next flight between two cities.
            This includes the name of the airline, flight number and the date and time
            of the next flight""",
        "parameters": {
            "type": "object",
            "properties": {
                "origin_city": {
                    "type": "string",
                    "description": "The name of the city where the flight originates",
                },
                "destination_city": {
                    "type": "string",
                    "description": "The flight destination city",
                },
            },
            "required": ["origin_city", "destination_city"],
        },
    },
}


messages = [
    SystemMessage(content="You an assistant that helps users find flight information."),
    UserMessage(
        content="I'm interested in going to Miami. What is the next flight there from Seattle?"
    ),
]

response = client.complete(
    messages=messages,
    tools=[tool],
    model=model_name,
)

# We expect the model to ask for a tool call
if response.choices[0].finish_reason == "tool_calls":

    # Append the model response to the chat history
    messages.append(response.choices[0].message)

    # We expect a single tool call
    if (
        response.choices[0].message.tool_calls
        and len(response.choices[0].message.tool_calls) == 1
    ):

        tool_call = response.choices[0].message.tool_calls[0]

        # We expect the tool to be a function call
        if tool_call.type == "function":

            # Parse the function call arguments and call the function
            function_args = json.loads(tool_call.function.arguments.replace("'", '"'))
            print(
                f"Calling function `{tool_call.function.name}` with arguments {function_args}"
            )
            callable_func = locals()[tool_call.function.name]
            function_return = callable_func(**function_args)
            print(f"Function returned = {function_return}")

            # Append the function call result fo the chat history
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": tool_call.function.name,
                    "content": function_return,
                }
            )

            # Get another response from the model
            response = client.complete(
                messages=messages,
                tools=[tool],
                model=model_name,
            )

            print(f"Model response = {response.choices[0].message.content}")

Calling function `get_flight_info` with arguments {'origin_city': 'Seattle', 'destination_city': 'Miami'}
Function returned = {"airline": "Iberia", "flight_number": "IL123", "flight_date": "May 12th, 2023", "flight_time": "10:10AM"}
Model response = The next flight from Seattle to Miami is with Iberia, flight number IL123. It is scheduled for May 12th, 2023, at 10:10 AM.


## Next Steps

To learn more about what you can do with the GitHub Models using Python, please check out the multiple [python cookbooks](../../../cookbooks/python/README.md).

For additional information about Azure AI Inference SDK, see full [documentation](https://aka.ms/azsdk/azure-ai-inference/python/reference) and [samples](https://aka.ms/azsdk/azure-ai-inference/python/samples).